In [69]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import math

In [ ]:
def read_timeseries_data(subnum, dataloc="../tseries", runtype="pair", runnum=1):
    """    
    Parameters:
      subnum: subject ID
      dataloc: location of timeseries and metadata
      runtype: either paired or random, type of run in task
      runnum: run number. Either 1 or 2 for random runs, or 1-6 for paired runs
   
    Returns:
      df: timeseries dataframe
    """
    fname = f"{dataloc}/sub-{subnum}_task-{runtype}_run-{runnum}_space-fsLR_den-91k_bold_timeseries.tsv"
    data = pd.read_csv(fname, sep="\t")
    return data

def read_metadata(subnum, dataloc="../tseries", runtype="pair", runnum=1):
    """
    Parameters:
      subnum: subject ID
      dataloc: location of timeseries and metadata
      runtype: either paired or random, type of run in task
      runnum: run number. Either 1 or 2 for random runs, or 1-6 for paired runs
    
    Returns:
      df: metadata dataframe  
    """
    fname = f"{dataloc}/sub-{subnum}_task-{runtype}_run-0{runnum}_events.tsv"
    data = pd.read_csv(fname, sep="\t")
    return data

def find_image_pairs(subnum="01"):
    """
    The current function finds the image pair ID's used throughout the visual statistical learning task
    
    Parameters:
      subnum: subject ID
      
    Returns:
      list: list of image pairs
    """
    meta = read_metadata(subnum)
    image_pairs = meta["trial_type"].unique()
    return image_pairs

def reducing_metadata_pulling_timeseries(image_of_interest, dataloc="../tseries", runtype="pair", runnum=1, subnum="01"):
    """
    The current function outputs a dataframe for 
    
    Parameters:
      image_of_interest: image ID
      dataloc: location of timeseries and metadata
      runtype: either paired or random, type of run in task
      subnum: subject ID
    
    Returns:
    
    """
    onsets = []
    #making a weights dictionary, where:
    ##if the time is 0: we don't need to weight anything out, hence 1
    ##if the time is 0.5: we create a weighting that takes 2 thirds of the first TR and 1 third of the second
    ##if the time is 1: we create a weighting that takes 1 third of the first TR and 2 thirds of the second
    weights_dict = dict(zip([0, 0.5, 1], [[1], [2/3, 1/3], [1/3, 2/3]]))

    meta = read_metadata(subnum, dataloc=dataloc, runtype=runtype, runnum=runnum)
    tseries = read_timeseries_data(subnum, dataloc=dataloc, runtype=runtype, runnum=runnum)
    img_rows = meta[meta['trial_type'] == image_of_interest]
    onset_time = img_rows["onset"].to_list()
    appended_data = []
    
    for i in onset_time:
        counter = 0
        TR = (i/1.5) + 3
        var = i%1.5 
        TR = math.floor(TR)
        rows = [TR] if var == 0 else [TR,TR+1]
        weights = weights_dict[var]
        for row, weight in zip(rows, weights):
            trow = tseries.iloc[int(row)] * weight 
            trow = pd.DataFrame(trow).transpose()
            TROW = trow if counter == 0 else pd.concat([TROW, trow])
            counter += 1
        TROW = TROW.sum(axis=0).to_frame().transpose()
        appended_data.append(TROW)
    appended_data = pd.concat(appended_data)
    return appended_data

def train_and_test(subnum, train=[["random", 1], ["pair", 5]], test=[["pair", 6]], images=["c11-5_71A", "c11-5_71B"]):
    appended_data = []
    appended_labels = []
    for runtype, runnum in train:
        for image in images:
            train_data = reducing_metadata_pulling_timeseries(image_of_interest = image, runnum=runnum, runtype=runtype, subnum=subnum)  
            appended_data.append(train_data)
            appended_labels.extend([image]*train_data.shape[0])
            
    appended_data = pd.concat(appended_data)

    appended_data_test = []
    appended_labels_test = []
    for runtype, runnum in test:
        for image in images:
            test_data = reducing_metadata_pulling_timeseries(image_of_interest = image, runnum=runnum, runtype=runtype, subnum=subnum)  
            appended_data_test.append(test_data)
            appended_labels_test.extend([image]*test_data.shape[0])
            
    appended_data_test = pd.concat(appended_data_test)
    return np.array(appended_data_test), np.array(appended_labels_test), np.array(appended_data), np.array(appended_labels)
  
def classifier():
    
    subid = ['01', '02', '03', '05', '06', '07', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '35', '36', '37']
    image = find_image_pairs()
    image = [[image[0], image[1]], [image[2], image[3]], [image[4], image[5]], [image[6], image[7]], [image[8], image[9]], [image[10], image[11]], [image[12], image[13]], [image[14], image[15]]]
    print(image)
    
    for sub in subid:
        for pos, i in enumerate(image):
            TESTX, TESTY, TRAINX, TRAINY = train_and_test(sub, images=i)
            clf.fit(TRAINX, TRAINY)
            acc = clf.score(TESTX, TESTY)
            x = clf.predict(TESTX)
            y = clf.predict_proba(TESTX)
            print(f"Classification Accuracyfor sub-{sub} & image pair {pos+1}: {acc}")
            col1 = np.average(y[:5, :1])
            col2 = np.average(y[:5, 1:])
            print(f"Classifier Evidence for sub-{sub} & image pair {pos+1}: {col1} {col2}")

classifier()

[['c47-34_29A', 'c47-34_29B'], ['c11-5_71A', 'c11-5_71B'], ['c79-97_86A', 'c79-97_86B'], ['c102-109_57A', 'c102-109_57B'], ['c56-68_0A', 'c56-68_0B'], ['c17-85_14A', 'c17-85_14B'], ['c83-101_100A', 'c83-101_100B'], ['c40-20_43A', 'c40-20_43B']]
Classification Accuracyfor sub-01 & image pair 1: 0.5
Classifier Evidence for sub-01 & image pair 1: 0.2751454984105615 0.7248545015894384
Classification Accuracyfor sub-01 & image pair 2: 0.7
Classifier Evidence for sub-01 & image pair 2: 0.6745141923953321 0.32548580760466805
Classification Accuracyfor sub-01 & image pair 3: 0.6
Classifier Evidence for sub-01 & image pair 3: 0.31797514307145935 0.6820248569285405
Classification Accuracyfor sub-01 & image pair 4: 0.6
Classifier Evidence for sub-01 & image pair 4: 0.25263726012934196 0.7473627398706582
Classification Accuracyfor sub-01 & image pair 5: 0.8
Classifier Evidence for sub-01 & image pair 5: 0.7765319382244134 0.2234680617755865
Classification Accuracyfor sub-01 & image pair 6: 0.5
Cla

Classification Accuracyfor sub-11 & image pair 4: 0.6
Classifier Evidence for sub-11 & image pair 4: 0.22452501565413607 0.775474984345864
Classification Accuracyfor sub-11 & image pair 5: 0.4
Classifier Evidence for sub-11 & image pair 5: 0.8645815597455782 0.13541844025442187


In [ ]:
#for each subject, for each image pair in the first and last learning runs, what is the classfiication accuracy (x), what is the classifer evidence (avergae of column 1 first 5, column 2 second 5)

#For every subject
##For each image pair in the first and last learning runs
###What is the classification accuracy
###What is the classifer evidence (avergae of column 1 first 5, column 2 second 5

In [122]:
image = find_image_pairs()
image = [[image[0], image[1]], [image[2], image[3]], [image[4], image[5]], [image[6], image[7]], [image[8], image[9]], [image[10], image[11]], [image[12], image[13]], [image[14], image[15]]]
print(image)

[['c47-34_29A', 'c47-34_29B'], ['c11-5_71A', 'c11-5_71B'], ['c79-97_86A', 'c79-97_86B'], ['c102-109_57A', 'c102-109_57B'], ['c56-68_0A', 'c56-68_0B'], ['c17-85_14A', 'c17-85_14B'], ['c83-101_100A', 'c83-101_100B'], ['c40-20_43A', 'c40-20_43B']]


In [128]:
for pos, i in enumerate(image):
    print(i)
    print(pos+1)

['c47-34_29A', 'c47-34_29B']
1
['c11-5_71A', 'c11-5_71B']
2
['c79-97_86A', 'c79-97_86B']
3
['c102-109_57A', 'c102-109_57B']
4
['c56-68_0A', 'c56-68_0B']
5
['c17-85_14A', 'c17-85_14B']
6
['c83-101_100A', 'c83-101_100B']
7
['c40-20_43A', 'c40-20_43B']
8


In [ ]:
for idx, x in enumerate(xs):
    print(idx, x)